# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments  \
0  Jun 14 2023  3:24PM  264774        10      38528.1   
1  Jun 14 2023  3:24PM  264774        10      8011005   
2  Jun 14 2023  3:24PM  264774        10      8011006   
3  Jun 14 2023  3:24PM  264774        10      8011007   
4  Jun 14 2023  3:18PM  264773        10     EYE43101   
5  Jun 14 2023  3:15PM  264772        16      9573025   
6  Jun 14 2023  3:14PM  264771        19  ADV80021127   
7  Jun 14 2023  3:14PM  264771        19  ADV80021123   
8  Jun 14 2023  3:14PM  264771        19  ADV80021124   
9  Jun 14 2023  3:14PM  264771        19  ADV80021125   

                         Customer ShipmentStatus  
0             Digital Brands, LLC       Released  
1             Digital Brands, LLC       Released  
2             Digital Brands, LLC       Released  
3             Digital Brands, LLC       Released  
4                  Eye Pharma Inc       Released  
5  Sartorius Bioprocess Solutions       Released  
6             AdvaGen Pharma, Ltd       Released  
7             AdvaGen Pharma, Ltd       Released  
8             AdvaGen Pharma, Ltd      Executing  
9             AdvaGen Pharma, Ltd       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
25  264771      Executing          1
26  264771       Released          4
27  264772       Released          1
28  264773       Released          1
29  264774       Released          4

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
264770                NaN        NaN       6.0
264771                NaN        1.0       4.0
264772                NaN        NaN       1.0
264773                NaN        NaN       1.0
264774                NaN        NaN       4.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
264703                0.0        0.0       1.0
264707               13.0        6.0       0.0
264713                0.0        0.0       2.0
264726                0.0        0.0       3.0
264733                0.0        0.0      13.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
264703                  0          0         1
264707                 13          6         0
264713                  0          0         2
264726                  0          0         3
264733                  0          0        13

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               264703          0          0         1
1               264707         13          6         0
2               264713          0          0         2
3               264726          0          0         3
4               264733          0          0        13

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               264703                            1
1               264707        13         6         
2               264713                            2
3               264726                            3
4               264733                           13

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Jun 14 2023  3:24PM  264774        10             Digital Brands, LLC
4   Jun 14 2023  3:18PM  264773        10                  Eye Pharma Inc
5   Jun 14 2023  3:15PM  264772        16  Sartorius Bioprocess Solutions
6   Jun 14 2023  3:14PM  264771        19             AdvaGen Pharma, Ltd
11  Jun 14 2023  3:10PM  264770        10                Methapharm, Inc.
17  Jun 14 2023  3:04PM  264768        10                         Bio-PRF
18  Jun 14 2023  3:03PM  264769        10                     Emerginnova
20  Jun 14 2023  2:48PM  264767        10    Snap Medical Industries, LLC
21  Jun 14 2023  2:11PM  264762        10                Methapharm, Inc.
23  Jun 14 2023  1:30PM  264759        10                     Emerginnova

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Jun 14 2023  3:24PM  264774        10             Digital Brands, LLC   
1  Jun 14 2023  3:18PM  264773        10                  Eye Pharma Inc   
2  Jun 14 2023  3:15PM  264772        16  Sartorius Bioprocess Solutions   
3  Jun 14 2023  3:14PM  264771        19             AdvaGen Pharma, Ltd   
4  Jun 14 2023  3:10PM  264770        10                Methapharm, Inc.   
5  Jun 14 2023  3:04PM  264768        10                         Bio-PRF   
6  Jun 14 2023  3:03PM  264769        10                     Emerginnova   
7  Jun 14 2023  2:48PM  264767        10    Snap Medical Industries, LLC   
8  Jun 14 2023  2:11PM  264762        10                Methapharm, Inc.   
9  Jun 14 2023  1:30PM  264759        10                     Emerginnova   

  Completed Executing Released  
0                            4  
1                            1  
2                            1  
3                   1        4  
4                            6  
5                            1  
6                            2  
7                            1  
8                            2  
9                            3

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Jun 14 2023  3:24PM  264774        10             Digital Brands, LLC   
1  Jun 14 2023  3:18PM  264773        10                  Eye Pharma Inc   
2  Jun 14 2023  3:15PM  264772        16  Sartorius Bioprocess Solutions   
3  Jun 14 2023  3:14PM  264771        19             AdvaGen Pharma, Ltd   
4  Jun 14 2023  3:10PM  264770        10                Methapharm, Inc.   
5  Jun 14 2023  3:04PM  264768        10                         Bio-PRF   
6  Jun 14 2023  3:03PM  264769        10                     Emerginnova   
7  Jun 14 2023  2:48PM  264767        10    Snap Medical Industries, LLC   
8  Jun 14 2023  2:11PM  264762        10                Methapharm, Inc.   
9  Jun 14 2023  1:30PM  264759        10                     Emerginnova   

  Released Executing Completed  
0        4                      
1        1                      
2        1                      
3        4         1            
4        6                      
5        1                      
6        2                      
7        1                      
8        2                      
9        3

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Jun 14 2023  3:24PM  264774        10             Digital Brands, LLC   
1  Jun 14 2023  3:18PM  264773        10                  Eye Pharma Inc   
2  Jun 14 2023  3:15PM  264772        16  Sartorius Bioprocess Solutions   
3  Jun 14 2023  3:14PM  264771        19             AdvaGen Pharma, Ltd   
4  Jun 14 2023  3:10PM  264770        10                Methapharm, Inc.   

  Released Executing Completed  
0        4                      
1        1                      
2        1                      
3        4         1            
4        6

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Jun 14 2023  3:24PM  264774        10             Digital Brands, LLC   
1  Jun 14 2023  3:18PM  264773        10                  Eye Pharma Inc   
2  Jun 14 2023  3:15PM  264772        16  Sartorius Bioprocess Solutions   
3  Jun 14 2023  3:14PM  264771        19             AdvaGen Pharma, Ltd   
4  Jun 14 2023  3:10PM  264770        10                Methapharm, Inc.   

   Released  Executing  Completed  
0       4.0        NaN        NaN  
1       1.0        NaN        NaN  
2       1.0        NaN        NaN  
3       4.0        1.0        NaN  
4       6.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Jun 14 2023  3:24PM  264774        10             Digital Brands, LLC   
1  Jun 14 2023  3:18PM  264773        10                  Eye Pharma Inc   
2  Jun 14 2023  3:15PM  264772        16  Sartorius Bioprocess Solutions   
3  Jun 14 2023  3:14PM  264771        19             AdvaGen Pharma, Ltd   
4  Jun 14 2023  3:10PM  264770        10                Methapharm, Inc.   

   Released  Executing  Completed  
0       4.0        0.0        0.0  
1       1.0        0.0        0.0  
2       1.0        0.0        0.0  
3       4.0        1.0        0.0  
4       6.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3971394      52.0        0.0        0.0
15          529504      16.0       30.0        0.0
16          264772       1.0        0.0        0.0
19         1588413       7.0        8.0       13.0
20          794196      29.0        0.0        0.0
21          264746       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3971394      52.0        0.0        0.0
1        15   529504      16.0       30.0        0.0
2        16   264772       1.0        0.0        0.0
3        19  1588413       7.0        8.0       13.0
4        20   794196      29.0        0.0        0.0
5        21   264746       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      52.0        0.0        0.0
1        15      16.0       30.0        0.0
2        16       1.0        0.0        0.0
3        19       7.0        8.0       13.0
4        20      29.0        0.0        0.0
5        21       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   52.0
1        15  Released   16.0
2        16  Released    1.0
3        19  Released    7.0
4        20  Released   29.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   16    19    20   21
Status                                     
Completed   0.0   0.0  0.0  13.0   0.0  0.0
Executing   0.0  30.0  0.0   8.0   0.0  0.0
Released   52.0  16.0  1.0   7.0  29.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   16    19    20   21
0          Completed   0.0   0.0  0.0  13.0   0.0  0.0
1          Executing   0.0  30.0  0.0   8.0   0.0  0.0
2           Released  52.0  16.0  1.0   7.0  29.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   16    19    20   21
0  Completed   0.0   0.0  0.0  13.0   0.0  0.0
1  Executing   0.0  30.0  0.0   8.0   0.0  0.0
2   Released  52.0  16.0  1.0   7.0  29.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()